# Estimate modulation transfer function of a screen

We base this on the procedure detailed in Tkačik G, Garrigan P, Ratliff C, Milčinski G, Klein JM, Seyfarth LH, et al. (2011) Natural Images from the Birthplace of the Human Eye. PLoS ONE 6(6): e20409. https://doi.org/10.1371/journal.pone.0020409.

We do this by porting over some of the MATLAB functions contained within the [UPENNNaturalImageProject](https://github.com/BrainardLab/UPENNNaturalImageProject) and [BrainardLabToolbox](https://github.com/BrainardLab/BrainardLabToolbox) github repositories. In their work, Tkačik et al use a Nikon D70 camera. We use a Nikon D90 camera, which is very similar. 

The following steps are taken:

1. Take a picture of the grating (this should be done with ISO 1600, f number 5.6, autofocus mode AF-S, and the camera set to vary the shutter speed).

2. Convert the NEF (raw image) to PGM using `dcraw`. This should be done with `-d` and `-4` flags, so that the command is `dcraw -d -4 DSC_*.NEF`. Note that in the text of the paper above, Tkačik et al say this will produce a PPM image. The authors note that the behavior of this command depends heavily on the version of `dcraw`. They use v5.71, I use v9.21, the version found in the Ubuntu PPA as of July 20, 2018 (so dcraw was installed by calling `sudo apt install dcraw` without adding any additional PPAs). In this case, the output of the file is a PGM, because it does not separate the R, G, and B channels (without the `-d` flag, a PPM is produced, separating the three channels). However, based on the description in the text, I believe the PGM is the proper image: it does not perform any interpolation and the authors later perform the demosaicing themselves, which doesn't make sense if the channels have already been separated. (The code also says that PGM images are produced, so maybe the references to PPM in the text of the article are a typo)

3. Demosaic the image to get the raw camera RGB values. This is done using code ported over from the BrainardLabToolbox/D70Toolbox. See the paper for more details.

4. Convert the raw RGB values to luminance values. This is done first by converting the RGB values to LMS absorptions, and then converting those values to luminance. To convert from RGB to LMS, we need the spectral sensitivities of the camera's R, G, and B sensors. We use both the values provided by Tkačik et al for the D70 and the values for the D90 found at the [Camera Spectral Sensitivity Database](http://www.gujinwei.org/research/camspec/db.html). We then convert to LMS using the weighted sum presented in the paper. (Note that Tkačik et al estimate the MTF separately for the R, G, and B sensors of the camera. Since we are only interested in the MTF of greyscale images, we convert to luminance and calculate the MTF from there.)

5. Open the resulting greyscale image in Inkscape, Photoshop, or similar softare, and identify the pixel locations necessary to create masks selecting the grating and the larger full contrast bars surrounding it. To do this, estimate the pixel location at the center of the grating and some pixels to give you a radius of the grating and of the mask.

6. Perform Fourier analysis? Or just plot a slice through and see for yourself?

We do NOT:

1. subtract the dark current. we hope that having the high contrast larger bars around the grating of interest mean this isn't necessary (any reduction in contrast due to dark current will show up here as well, so we just compare the grating to these bars).

2. standardize the RGB values. since we're always taking pictures using the same parameters, this isn't necessary.

SHOULD I STANDARDIZE the RGB values? Because the shutter speed might vary. Or should I set the shutter speed to be the same?

In [7]:
import numpy as np
from scipy import misc

Create the PGM images:

In [6]:
%%bash
dcraw -4 -d data/DSC_*.NEF

Demosaic the images. This is done in two parts:

1.  Set up mosaic mask and produce a three channel image where each channel is masked according to the sensor mosaic. The sensor mosaic the authors say is used by the D70 is
```
B  G
G  R
```
However, it appears that the sensor mosaic used by the D90 is
```
G  B
R  G
```
At least, according to [two](http://www.silios.gr/2015/06/nikon-d90-in-astrophotography-sensor-specifications/) [sources](https://sites.textiles.ncsu.edu/color-science-lab/current-research/camera-colorimetry) I found. I'm not sure the best way to check this.

2. Block average the image. This performs a weighted average on the image in 2x2 pixel blocks, weighting the red and blue pixels by 4 and the green pixels by 2 to make up for the fact that the green pixels are over-represented on the sensor mosaic.

In [15]:
im = misc.imread('data/DSC_0004.pgm').astype(np.double)

/home/billbrod/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  if __name__ == '__main__':


In [117]:
# We "pythonify" the conventions in the Tkačik et al paper: R=0, G=1, B=2
D90_BAYER = np.array([[1, 2], [0, 1]])
def create_mosaic_mask(bayer_mosaic, height, width):
    """create mask based on Bayer mosaic of the camera
    
    this is based on the Brainard's lab's SimToolbox's SimCreateMask.m"""
    # First create a small kernel representing the RGB mosaic pattern
    kernels = np.zeros((len(np.unique(bayer_mosaic)), bayer_mosaic.shape[0], bayer_mosaic.shape[1]))
    for i in range(bayer_mosaic.shape[0]):
        for j in range(bayer_mosaic.shape[1]):
            kernels[bayer_mosaic[i, j], i, j] = 1

    # Then repeat the kernel to build a mask to the size  of the image
    mask = []
    for n in range(kernels.shape[0]):
        mask.append(np.tile(kernels[n], (int(np.ceil(height / float(bayer_mosaic.shape[0]))+1), int(np.ceil(width / float(bayer_mosaic.shape[1]))+1))))
    mask = np.dstack(mask).swapaxes(0, 2)
    if mask.shape[1] < height or mask.shape[2] < width:
        raise Exception('Logic error in computation of mask size')
    mask = mask[:, 1:height, 1:width]
    return mask, kernels

In [118]:
mask, kernels = create_mosaic_mask(D90_BAYER, 10, 10)

... some other stuff, but skipping ahead to get this here:

to convert from RGB to LMS, can either use the conversion matrix given by the paper:
```
   1.0e-07 *

    0.1267    0.2467   -0.0450
    0.0327    0.2904   -0.0299
    0.0038   -0.0292    0.2425
```

or grab the RGB sensitivites from the [database](http://www.gujinwei.org/research/camspec/db.html) (`S_RGB`) and the cone fundamentals from [here](http://www.cvrl.org/cones.htm) (2 degree fundamentals, linear energy, 5 nm, csv; `S_LMS`) and do the following (this is matlab code, update it). The first bit is because the `S_LMS` includes values below 400 and above 720, which is what `S_RGB` cover, and because `S_LMS` is sampled in 5nm increments, while `S_RGB` is sampled in 10nm ones. Note that for the below to work, both sensitivities have to be 31x3, so transpose them if that's not the case.

```
S_LMS=S_LMS(S_LMS(:,1)>=400,:)
S_LMS=S_LMS(S_LMS(:,1)<=720,:)
S_LMS=S_LMS(1:2:end,2:4)
(S_RGB \ S_LMS)'
```


for LMS to luminance, use the one provided by the paper:
```
433.9441  275.8200   -0.0935
```

Both of the matrices get multiplied by the observed values from the right (so the pixel values should be 3xN)